It happens all the time: someone gives you data containing malformed strings,
Python, lists and missing data. How do you tidy it up so you can get on with the
analysis?

Take this monstrosity as the DataFrame to use in the following puzzles:

df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN',
'londON_StockhOlm',
'Budapest_PaRis', 'Brussels_londOn'],
'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )',
'12. Air France', '"Swiss Air"']})

1. Some values in the the FlightNumber column are missing. These numbers are
meant to increase by 10 with each row so 10055 and 10075 need to be put in
place. Fill in these missing numbers and make the column an integer column
(instead of a float column).
    
2. The From_To column would be better as two separate columns! Split each
string on the underscore delimiter _ to give a new temporary DataFrame with
the correct values. Assign the correct column names to this temporary
DataFrame.
    
3. Notice how the capitalisation of the city names is all mixed up in this
temporary DataFrame. Standardise the strings so that only the first letter is
uppercase (e.g. "londON" should become "London".)
    
4. Delete the From_To column from df and attach the temporary DataFrame
from the previous questions.
    
5. In the RecentDelays column, the values have been entered into the
DataFrame as a list. We would like each first value in its own column, each

second value in its own column, and so on. If there isn't an Nth value, the value
should be NaN.
    
Expand the Series of lists into a DataFrame named delays, rename the columns
delay_1, delay_2, etc. and replace the unwanted RecentDelays column in df
with delays.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN',
'londON_StockhOlm',

'Budapest_PaRis', 'Brussels_londOn'],
'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )',
'12. Air France', '"Swiss Air"']})

In [3]:
print(df.head())

            From_To  FlightNumber  RecentDelays              Airline
0      LoNDon_paris       10045.0      [23, 47]               KLM(!)
1      MAdrid_miLAN           NaN            []    <Air France> (12)
2  londON_StockhOlm       10065.0  [24, 43, 87]  (British Airways. )
3    Budapest_PaRis           NaN          [13]       12. Air France
4   Brussels_londOn       10085.0      [67, 32]          "Swiss Air"


In [4]:
# Filling missing values
missing_values = [10055, 10075]
df.loc[df.FlightNumber.isnull(), 'FlightNumber'] = missing_values

In [5]:
# Converting FlightNumber type to integer
df.FlightNumber = pd.to_numeric(df.FlightNumber, downcast='integer')

In [6]:
df.dtypes

From_To         object
FlightNumber     int16
RecentDelays    object
Airline         object
dtype: object

In [7]:
print(df.head())

            From_To  FlightNumber  RecentDelays              Airline
0      LoNDon_paris         10045      [23, 47]               KLM(!)
1      MAdrid_miLAN         10055            []    <Air France> (12)
2  londON_StockhOlm         10065  [24, 43, 87]  (British Airways. )
3    Budapest_PaRis         10075          [13]       12. Air France
4   Brussels_londOn         10085      [67, 32]          "Swiss Air"


In [8]:
# Splitting and formatting From_To column to a temporary dataframe
names = ['From', 'To']
temp_df = pd.DataFrame(list(df.From_To.str.lower().str.split('_')), columns=names)
temp_df.From = temp_df.From.str.capitalize()
temp_df.To = temp_df.To.str.capitalize()

In [9]:
print(temp_df.head())

       From         To
0    London      Paris
1    Madrid      Milan
2    London  Stockholm
3  Budapest      Paris
4  Brussels     London


In [10]:
# Joining dataframes and dropping unnecessary From_To column
df = temp_df.join(df)
df.drop('From_To', axis=1, inplace=True)

In [11]:
print(df.head())

       From         To  FlightNumber  RecentDelays              Airline
0    London      Paris         10045      [23, 47]               KLM(!)
1    Madrid      Milan         10055            []    <Air France> (12)
2    London  Stockholm         10065  [24, 43, 87]  (British Airways. )
3  Budapest      Paris         10075          [13]       12. Air France
4  Brussels     London         10085      [67, 32]          "Swiss Air"


In [12]:
# Defining the max length of list in RecetDelays column
max_len = 0
for item in list(df.RecentDelays):
    if len(item) > max_len:
        max_len = len(item)
# Filling lists to have equal number of elements
# Getting rid of the warning about modifying a copy of slice since it is intentional
pd.options.mode.chained_assignment = None 
for i, item in enumerate(list(df.RecentDelays)):
    temp = item.copy()
    while len(temp) < max_len:
        temp.append(np.nan)
    df.RecentDelays[i] = temp

In [13]:
print(df.head())

       From         To  FlightNumber     RecentDelays              Airline
0    London      Paris         10045    [23, 47, nan]               KLM(!)
1    Madrid      Milan         10055  [nan, nan, nan]    <Air France> (12)
2    London  Stockholm         10065     [24, 43, 87]  (British Airways. )
3  Budapest      Paris         10075   [13, nan, nan]       12. Air France
4  Brussels     London         10085    [67, 32, nan]          "Swiss Air"


In [14]:
# Making list of titles for the new columns
delays = ['delay_{}'.format(i+1) for i in range(max_len)]
print(delays)
# Inserting new delays into df and dropping RecentDelays column
df[delays] = pd.DataFrame(df.RecentDelays.values.tolist())
df.drop('RecentDelays', axis=1, inplace=True)

['delay_1', 'delay_2', 'delay_3']


In [15]:
print(df.head())
# I just realized that all the steps with filling lists with NaNs weren't necessary
# I could have just done
# df[['delay_1', 'delay_2', 'delay_3']] = pd.DataFrame(df.RecentDelays.values.tolist())
# or at least comprehense the numbers on delay_1, delay_2 etc with max_len of RecentDelays list

       From         To  FlightNumber              Airline  delay_1  delay_2  \
0    London      Paris         10045               KLM(!)     23.0     47.0   
1    Madrid      Milan         10055    <Air France> (12)      NaN      NaN   
2    London  Stockholm         10065  (British Airways. )     24.0     43.0   
3  Budapest      Paris         10075       12. Air France     13.0      NaN   
4  Brussels     London         10085          "Swiss Air"     67.0     32.0   

   delay_3  
0      NaN  
1      NaN  
2     87.0  
3      NaN  
4      NaN  


In [16]:
# Rearranging order of columns
cols = list(df.columns)
print(cols)

['From', 'To', 'FlightNumber', 'Airline', 'delay_1', 'delay_2', 'delay_3']


In [17]:
cols = ['From', 'To', 'FlightNumber', 'delay_1', 'delay_2', 'delay_3', 'Airline']
df = df[cols]

In [18]:
print(df.head())

       From         To  FlightNumber  delay_1  delay_2  delay_3  \
0    London      Paris         10045     23.0     47.0      NaN   
1    Madrid      Milan         10055      NaN      NaN      NaN   
2    London  Stockholm         10065     24.0     43.0     87.0   
3  Budapest      Paris         10075     13.0      NaN      NaN   
4  Brussels     London         10085     67.0     32.0      NaN   

               Airline  
0               KLM(!)  
1    <Air France> (12)  
2  (British Airways. )  
3       12. Air France  
4          "Swiss Air"  


In [19]:
# Cleaning the Airline column
values = df.Airline
values = values.str.replace('(', '').str.replace(')', '').str.replace('!', '').str.replace('.', '') \
        .str.replace('"', '').str.replace('<', '').str.replace('>', '').str.replace('1', '') \
        .str.replace('2', '').str.strip()
df.Airline = values

In [20]:
print(df.head())

       From         To  FlightNumber  delay_1  delay_2  delay_3  \
0    London      Paris         10045     23.0     47.0      NaN   
1    Madrid      Milan         10055      NaN      NaN      NaN   
2    London  Stockholm         10065     24.0     43.0     87.0   
3  Budapest      Paris         10075     13.0      NaN      NaN   
4  Brussels     London         10085     67.0     32.0      NaN   

           Airline  
0              KLM  
1       Air France  
2  British Airways  
3       Air France  
4        Swiss Air  
